# Amplifier Modules: Building Blocks of the Agent System

This notebook explains the Amplifier module system - how modules work, the six module types, and how to create your own modules.

## Important: amplifier-core vs amplifierd

**This notebook covers amplifier-core concepts** - the execution runtime and module system.

**amplifierd** is the REST API daemon that:
- Manages profiles and collections
- Resolves module sources from git URLs and other sources
- Generates mount plans (dict-based specifications)
- Provides API for session management

**The concepts here apply to the execution layer.** In production, module loading happens through:
- REST API (`POST /api/v1/sessions/` generates mount plan)
- Profile compilation (refs resolved to cache paths)
- Mount plans (dict-based specifications with file:// URIs)

## Philosophy: Linux Kernel Inspired

Amplifier follows a modular architecture inspired by the Linux Kernel:

- **Kernel (amplifier-core)**: Ultra-thin, stable, provides MECHANISMS only
- **Modules**: Self-contained "bricks" with stable interface "studs"
- **Principle**: "The center stays still so the edges can move fast"

### Key Concepts

**Mechanism vs Policy**:
- **Mechanism** (kernel): *How* to load, coordinate, execute modules
- **Policy** (modules): *Which* provider, *what* tools, *when* to use them

**Non-Interference**:
- Failing module cannot crash the kernel
- Modules are isolated from each other
- Graceful degradation always

## Part 1: The Six Module Types

Amplifier has six module types, each serving a specific purpose:

| Type | Purpose | Cardinality | Examples |
|------|---------|-------------|----------|
| **Provider** | LLM backend | Multiple | provider-anthropic, provider-openai |
| **Tool** | Capability | Multiple | tool-filesystem, tool-bash, tool-web |
| **Orchestrator** | Execution loop | Single | loop-streaming, loop-events |
| **Context** | Memory management | Single | context-simple, context-persistent |
| **Hook** | Observability | Multiple | hooks-logging, hooks-approval |
| **Agent** | Config overlays | N/A | Delegation configurations |

## Module Types Overview

Amplifier has six module types:

| Type | Purpose | Cardinality | Examples | Key Methods |
|------|---------|-------------|----------|-------------|
| **Provider** | LLM backend | Multiple | provider-anthropic, provider-openai | complete(), parse_tool_calls() |
| **Tool** | Agent capabilities | Multiple | tool-filesystem, tool-bash, tool-web | execute() |
| **Orchestrator** | Execution loop | Single | loop-basic, loop-streaming | execute() |
| **Context** | Memory management | Single | context-simple, context-persistent | add_message(), get_messages(), compact() |
| **Hook** | Observability | Multiple | hooks-logging, hooks-approval | __call__() |

## Part 2: Module Contracts (Protocols)

Each module type implements a Python protocol defining its interface. This ensures modules can be swapped without breaking the system.

**Important Note on Protocols**: These are interface contracts that define what methods modules must implement. The actual implementations may vary, but all modules must conform to these protocols. In production (amplifierd), mount plans reference modules via file:// paths after source resolution.

### Provider Protocol

In [ ]:
from dataclasses import dataclass
from typing import Any
from typing import Protocol

# Simplified versions of the actual protocols for demonstration


@dataclass
class ProviderResponse:
    """Response from LLM provider."""

    content: str
    content_blocks: list[dict]
    raw: Any


@dataclass
class ToolCall:
    """Parsed tool call from LLM."""

    id: str
    tool: str
    arguments: dict[str, Any]


class Provider(Protocol):
    """LLM backend interface."""

    async def complete(self, messages: list[dict], tools: list[Any] | None = None, **kwargs) -> ProviderResponse:
        """Generate completion from messages."""
        ...

    def parse_tool_calls(self, response: ProviderResponse) -> list[ToolCall]:
        """Extract tool calls from response."""
        ...


print("Provider Protocol:")
print("  - complete(): Send messages to LLM, get response")
print("  - parse_tool_calls(): Extract tool calls from LLM response")
print("\nExample providers: provider-anthropic, provider-openai")

### Tool Protocol

In [ ]:
@dataclass
class ToolResult:
    """Result from tool execution."""

    success: bool
    output: Any = None
    error: dict[str, Any] | None = None


class Tool(Protocol):
    """Agent capability interface."""

    async def execute(self, input: dict[str, Any]) -> ToolResult:
        """Execute tool with given input."""
        ...


print("Tool Protocol:")
print("  - execute(): Perform action with input, return result")
print("\nExample tools:")
print("  - tool-filesystem: Read, write, edit files")
print("  - tool-bash: Execute shell commands")
print("  - tool-web: Fetch web content")
print("  - tool-task: Delegate to specialized agents")

### Orchestrator Protocol

In [ ]:
class Orchestrator(Protocol):
    """Execution loop interface."""

    async def execute(
        self,
        prompt: str,
        context: Any,  # ContextManager
        providers: dict[str, Any],  # dict[str, Provider]
        tools: dict[str, Any],  # dict[str, Tool]
        hooks: Any,  # HookRegistry
        coordinator: Any | None = None,  # ModuleCoordinator
    ) -> str:
        """Execute prompt and return response."""
        ...


print("Orchestrator Protocol:")
print("  - execute(): Main agent loop - sends prompts, handles tool calls, returns response")
print("\nExample orchestrators:")
print("  - loop-basic: Simple request-response loop")
print("  - loop-streaming: Streaming responses with real-time tool execution")
print("  - loop-events: Event-driven orchestration with rich observability")

### Context Protocol

In [ ]:
class ContextManager(Protocol):
    """Memory management interface."""

    async def add_message(self, message: dict) -> None:
        """Add message to history."""
        ...

    async def get_messages(self) -> list[dict]:
        """Retrieve all messages."""
        ...

    async def should_compact(self) -> bool:
        """Check if compaction needed."""
        ...

    async def compact(self) -> None:
        """Compact message history."""
        ...

    async def clear(self) -> None:
        """Clear all messages."""
        ...


print("Context Protocol:")
print("  - add_message(): Add to conversation history")
print("  - get_messages(): Retrieve history for LLM")
print("  - should_compact(): Check if history too large")
print("  - compact(): Summarize/reduce history size")
print("  - clear(): Reset conversation")
print("\nExample context managers:")
print("  - context-simple: In-memory, no persistence")
print("  - context-persistent: Disk storage, automatic compaction")

### Hook Protocol

In [ ]:
@dataclass
class HookResult:
    """Result from hook handler."""

    action: str  # "continue", "block", "modify"
    data: dict[str, Any] | None = None


class HookHandler(Protocol):
    """Observability hook interface."""

    async def __call__(self, event: str, data: dict[str, Any]) -> HookResult:
        """Process event and return action."""
        ...


print("Hook Protocol:")
print("  - __call__(): React to system events (tool execution, LLM calls, etc.)")
print("\nExample hooks:")
print("  - hooks-logging: Log all events to file")
print("  - hooks-approval: Request human approval for actions")
print("  - hooks-status-context: Inject git/system status into prompts")
print("  - hooks-streaming-ui: Display real-time progress")

## Part 3: Module Resolution in Amplifierd

In production, modules are resolved through profile manifests, not Python entry points or environment variables.

### Manifest-Based Module Resolution

Profiles declare modules with explicit sources:

### Profile Module Declaration

Profiles declare modules in YAML frontmatter:

```yaml
---
profile:
  name: example
  version: 1.0.0

providers:
  - module: provider-anthropic
    source: git+https://github.com/microsoft/amplifier-module-provider-anthropic@main
    config:
      model: claude-sonnet-4-5

tools:
  - module: tool-filesystem
    source: git+https://github.com/microsoft/amplifier-module-tool-filesystem@main
    config:
      working_dir: /workspace
      
  - module: tool-bash  
    source: git+https://github.com/microsoft/amplifier-module-tool-bash@main
---
Profile instructions here...
```

### Source URI Formats

Modules can be loaded from multiple source types:

```yaml
# Git repository (most common)
source: git+https://github.com/org/repo@main
source: git+https://github.com/org/repo@v1.2.3
source: git+https://github.com/org/repo@commit-hash

# Git repository with subdirectory
source: git+https://github.com/org/monorepo@main#subdir=packages/tool-xyz

# Local file path (for development)
source: file:///path/to/module

# Embedded in profile (optional)
source: embedded
# Module code in profile's modules/ directory
```

**Format**: `{type}+{url}@{ref}#{fragment}`

### Resolution and Caching Process

The daemon handles module resolution:

**1. Profile Compilation**:
```
User requests session with profile
  ↓
Daemon loads profile YAML
  ↓
Parses module declarations with sources
  ↓
Generates mount plan
```

**2. Source Resolution**:
```
For git+ sources:
  ↓
Clone repository to temporary location
  ↓
Extract to cache: $AMPLIFIERD_HOME/cache/git/{commit}/
  ↓
Create file:// URI to cached location
```

**3. Mount Plan Generation**:
```python
{
    "modules": {
        "foundation/base.provider.anthropic": {
            "module": "provider-anthropic",
            "source": "file:///path/to/cache/git/abc123/",
            "config": {"model": "claude-sonnet-4-5"}
        },
        "foundation/base.tool.filesystem": {
            "module": "tool-filesystem", 
            "source": "file:///path/to/cache/git/def456/",
            "config": {"working_dir": "/workspace"}
        }
    }
}
```

**4. Runtime Loading**:
```
amplifier-core receives mount plan
  ↓
Loads modules from file:// paths
  ↓
Calls mount() functions
  ↓
Modules register with coordinator
```

### Module ID Format

In mount plans, modules are identified as: `{profile-path}.{type}.{name}`

Examples:
- `foundation/base.provider.anthropic`
- `foundation/base.tool.filesystem`
- `custom/agent.tool.database`

### Directory Structure

```
$AMPLIFIERD_HOME/
├── share/
│   └── profiles/{collection}/{profile}/
│       ├── {profile}.md           # Profile definition
│       ├── agents/                # Agent configs
│       ├── context/               # Context files
│       └── modules/               # Embedded modules (optional)
└── cache/
    └── git/
        ├── {commit}/              # Cached git module
        └── {commit}_{subdir}/     # Cached git subdir module
```

## Part 4: Module Mounting

Every module must implement a `mount()` function. This is the entry point the kernel uses to initialize the module.

**Note**: The mount() function is the same whether modules are loaded via entry points (development) or mount plans (production). The mounting mechanism is consistent - only the discovery differs.

### The mount() Function Signature

In [ ]:
from collections.abc import Callable


# Template for mount() function
async def mount(
    coordinator: Any,  # ModuleCoordinator
    config: dict[str, Any] | None = None,
) -> Callable | None:
    """
    Mount module and return cleanup function.

    Args:
        coordinator: Central coordination hub
        config: Module-specific configuration

    Returns:
        Optional cleanup function
    """
    # 1. Create module instance with config
    # module = MyModule(config or {})

    # 2. Mount to coordinator
    # await coordinator.mount("mount_point", module)

    # 3. Return cleanup function (optional)
    # def cleanup():
    #     # Close resources, unregister, etc.
    #     pass
    # return cleanup

    return None


print("The mount() function is responsible for:")
print("  1. Instantiating the module with configuration")
print("  2. Registering the module with the coordinator")
print("  3. Returning a cleanup function (if resources need cleanup)")
print("\nMount points:")
mount_points = {
    "orchestrator": "Single execution loop",
    "context": "Single memory manager",
    "providers": "Multiple LLM backends (dict)",
    "tools": "Multiple capabilities (dict)",
    "hooks": "Multiple observers (registry)",
}
for point, desc in mount_points.items():
    print(f"  - {point}: {desc}")

## Part 5: Creating a Custom Tool Module

Let's walk through creating a simple custom tool module step by step.

### Example: Currency Converter Tool

In [ ]:
# Step 1: Define the tool class
import logging

logger = logging.getLogger(__name__)


class CurrencyConverterTool:
    """A simple currency converter tool."""

    def __init__(self, config: dict[str, Any]):
        self.config = config
        # In real implementation, would load exchange rates
        self.rates = {"USD_EUR": 0.92, "EUR_USD": 1.09, "USD_GBP": 0.79, "GBP_USD": 1.27}

    async def execute(self, input: dict[str, Any]) -> ToolResult:
        """Convert currency amount."""
        try:
            amount = float(input.get("amount", 0))
            from_currency = input.get("from_currency", "USD")
            to_currency = input.get("to_currency", "EUR")

            rate_key = f"{from_currency}_{to_currency}"

            if rate_key not in self.rates:
                return ToolResult(success=False, error={"message": f"Conversion {rate_key} not supported"})

            rate = self.rates[rate_key]
            result = amount * rate

            return ToolResult(
                success=True,
                output={
                    "amount": result,
                    "currency": to_currency,
                    "rate": rate,
                    "original_amount": amount,
                    "original_currency": from_currency,
                },
            )
        except Exception as e:
            logger.error(f"Currency conversion failed: {e}")
            return ToolResult(success=False, error={"message": str(e)})


print("✓ Step 1: Tool class defined")
print("  - Implements execute() method")
print("  - Returns ToolResult with success/output or error")
print("  - Handles exceptions gracefully")

In [ ]:
# Step 2: Implement the mount() function
async def mount_currency_tool(coordinator: Any, config: dict[str, Any] | None = None) -> None:
    """
    Mount the currency converter tool.

    This would typically be in: amplifier_module_tool_currency/__init__.py
    """
    config = config or {}

    # Create tool instance
    CurrencyConverterTool(config)

    # Mount to coordinator
    # In real code: await coordinator.mount("tools", tool, name="currency")
    print(f"  Mounted CurrencyConverterTool with config: {config}")

    logger.info("Mounted CurrencyConverterTool")

    # No cleanup needed for this simple tool
    return


print("✓ Step 2: mount() function implemented")
print("  - Takes coordinator and optional config")
print("  - Creates tool instance")
print("  - Registers with coordinator")
print("  - Returns cleanup function (None in this case)")

### Step 3: Package Configuration

For modules published to GitHub, create `pyproject.toml`:

```toml
[project]
name = "amplifier-module-tool-currency"
version = "0.1.0"
dependencies = ["amplifier-core"]

[project.entry-points."amplifier.modules"]
tool-currency = "amplifier_module_tool_currency:mount"
```

**Key points:**
- Package name: `amplifier-module-tool-{name}`
- Depends on `amplifier-core`
- Registers entry point for local development/testing
- Entry point format: `{module-id} = "{package}:{mount-function}"`

**In production (amplifierd)**:
- Entry points are NOT used for discovery
- Modules referenced by `source:` URI in profiles
- Git repositories cloned and cached
- Mount function loaded from cached location

In [ ]:
# Step 4: Testing the tool


async def test_currency_tool():
    """Test the currency converter tool."""
    tool = CurrencyConverterTool({})

    # Test 1: Valid conversion
    result1 = await tool.execute({"amount": 100, "from_currency": "USD", "to_currency": "EUR"})
    print("Test 1 - Valid conversion (100 USD to EUR):")
    print(f"  Success: {result1.success}")
    print(f"  Output: {result1.output}")

    # Test 2: Invalid conversion
    result2 = await tool.execute(
        {
            "amount": 100,
            "from_currency": "USD",
            "to_currency": "JPY",  # Not in our rates
        }
    )
    print("\nTest 2 - Invalid conversion (USD to JPY):")
    print(f"  Success: {result2.success}")
    print(f"  Error: {result2.error}")


# Run the test
await test_currency_tool()

print("\n✓ Step 4: Tool tested successfully")

### Using the Custom Tool in a Profile

### Using the Custom Tool

Once published to GitHub, reference in a profile:

```yaml
---
profile:
  name: financial-assistant
  version: 1.0.0

providers:
  - module: provider-anthropic
    source: git+https://github.com/microsoft/amplifier-module-provider-anthropic@main
    config:
      model: claude-sonnet-4-5

tools:
  - module: tool-currency
    source: git+https://github.com/yourorg/amplifier-module-tool-currency@main
    config:
      update_interval: 3600  # Refresh rates every hour
---
You are a financial assistant with access to currency conversion.
```

**Resolution flow**:
1. User creates session with this profile
2. Daemon clones tool-currency repository
3. Extracts to cache: `$AMPLIFIERD_HOME/cache/git/{commit}/`
4. Generates mount plan with `file:///path/to/cache/git/{commit}/`
5. amplifier-core loads from cached location
6. Calls `mount(coordinator, config)`
7. Tool becomes available in session

**No entry points or package installation required** - modules are resolved dynamically from source URIs.

## Part 6: Creating a Provider Module

Provider modules are more complex. Let's look at a simplified example.

In [ ]:
# Simplified provider example
class MockProvider:
    """A mock provider for testing."""

    def __init__(self, config: dict[str, Any]):
        self.config = config
        self.default_model = config.get("default_model", "mock-model")

    async def complete(self, messages: list[dict], tools: list[Any] | None = None, **kwargs) -> ProviderResponse:
        """Generate mock completion."""
        # In real provider: call actual LLM API

        # Mock response
        content = f"Mock response to: {messages[-1].get('content', 'unknown')}"

        return ProviderResponse(
            content=content,
            content_blocks=[{"type": "text", "text": content}],
            raw={"model": self.default_model, "usage": {"tokens": 10}},
        )

    def parse_tool_calls(self, response: ProviderResponse) -> list[ToolCall]:
        """Extract tool calls from response."""
        # In real provider: parse tool calls from response
        # For mock: return empty list
        return []


async def mount_mock_provider(coordinator: Any, config: dict[str, Any] | None = None) -> None:
    """Mount the mock provider."""
    config = config or {}
    provider = MockProvider(config)

    # In real code: await coordinator.mount("providers", provider, name="mock")
    print(f"Mounted MockProvider with model: {provider.default_model}")

    return


print("Provider Module Structure:")
print("\n1. Implements complete() - core LLM interaction")
print("   - Takes messages (conversation history)")
print("   - Takes tools (available capabilities)")
print("   - Returns ProviderResponse")
print("\n2. Implements parse_tool_calls() - extract tool invocations")
print("   - Parses LLM response for tool calls")
print("   - Returns list of ToolCall objects")
print("\n3. Handles provider-specific details")
print("   - API authentication")
print("   - Message format conversion")
print("   - Tool schema translation")
print("   - Error handling")

## Part 7: Advanced - One Module, Multiple Tools

A single module can provide multiple tools. This is common for related functionality.

In [ ]:
# Example: Math tools module providing multiple operations
class AddTool:
    async def execute(self, input: dict[str, Any]) -> ToolResult:
        try:
            result = input["a"] + input["b"]
            return ToolResult(success=True, output={"result": result})
        except Exception as e:
            return ToolResult(success=False, error={"message": str(e)})


class MultiplyTool:
    async def execute(self, input: dict[str, Any]) -> ToolResult:
        try:
            result = input["a"] * input["b"]
            return ToolResult(success=True, output={"result": result})
        except Exception as e:
            return ToolResult(success=False, error={"message": str(e)})


async def mount_math_tools(coordinator: Any, config: dict[str, Any] | None = None) -> None:
    """Mount multiple math tools from one module."""
    config = config or {}

    # Mount multiple tools with different names
    # await coordinator.mount("tools", AddTool(config), name="math-add")
    # await coordinator.mount("tools", MultiplyTool(config), name="math-multiply")

    print("Mounted math tools:")
    print("  - math-add: Addition operation")
    print("  - math-multiply: Multiplication operation")

    logger.info("Mounted 2 math tools from single module")
    return


print("One Module, Multiple Tools:")
print("\nAdvantages:")
print("  ✓ Related functionality grouped together")
print("  ✓ Shared configuration and resources")
print("  ✓ Single source/version management (one git repo)")
print("  ✓ Easier testing and maintenance")
print("\nExample: tool-filesystem provides:")
print("  - Read: Read file contents")
print("  - Write: Write file contents")
print("  - Edit: Edit file in place")
print("  - Glob: Search for files by pattern")
print("  - Grep: Search file contents")
print("\nIn profile:")
profile_example = """
tools:
  - module: tool-math
    source: git+https://github.com/org/amplifier-module-tool-math@main
    # Provides both math-add and math-multiply
"""
print(profile_example)

## Part 8: Best Practices

### 1. Never Raise Exceptions from Protocol Methods

In [ ]:
# BAD: Unhandled exceptions
class BadTool:
    async def execute(self, input: dict[str, Any]) -> ToolResult:
        result = risky_operation()  # Could raise!
        return ToolResult(success=True, output=result)


# GOOD: Graceful error handling
class GoodTool:
    async def execute(self, input: dict[str, Any]) -> ToolResult:
        try:
            result = risky_operation()
            return ToolResult(success=True, output=result)
        except Exception as e:
            logger.error(f"Tool failed: {e}", exc_info=True)
            return ToolResult(success=False, error={"message": str(e), "type": type(e).__name__})


def risky_operation():
    raise ValueError("Something went wrong")


print("Best Practice #1: Never Raise Exceptions")
print("\nWhy?")
print("  - Exceptions can crash the kernel")
print("  - Breaks the module isolation guarantee")
print("  - LLM sees unhelpful error messages")
print("\nInstead:")
print("  ✓ Wrap all operations in try/except")
print("  ✓ Return ToolResult with success=False")
print("  ✓ Provide helpful error messages")
print("  ✓ Log the full exception for debugging")

### 2. Make Modules Configurable

In [ ]:
# BAD: Hardcoded values
class BadConfigTool:
    def __init__(self):
        self.timeout = 30  # Hardcoded
        self.retries = 3  # Hardcoded


# GOOD: Configurable with sensible defaults
class GoodConfigTool:
    def __init__(self, config: dict[str, Any]):
        self.timeout = config.get("timeout", 30)  # Default: 30
        self.retries = config.get("retries", 3)  # Default: 3
        self.debug = config.get("debug", False)  # Default: False

        # Validate config
        if self.timeout < 1:
            raise ValueError("timeout must be >= 1")
        if self.retries < 0:
            raise ValueError("retries must be >= 0")


print("Best Practice #2: Configurable Modules")
print("\nUsage in profile:")
profile_config_example = """
tools:
  - module: tool-example
    config:
      timeout: 60      # Override default
      retries: 5       # Override default
      debug: true      # Enable debug mode
"""
print(profile_config_example)
print("Benefits:")
print("  ✓ Users can customize behavior")
print("  ✓ Different environments (dev/prod) can use different settings")
print("  ✓ No code changes needed for tuning")

### 3. Clean Up Resources

In [ ]:
# Example: Tool with resources that need cleanup
class DatabaseTool:
    def __init__(self, config: dict[str, Any]):
        self.connection = None  # Database connection
        self.temp_files = []  # Temporary files

    async def connect(self):
        # Open database connection
        self.connection = "<database connection>"

    def cleanup(self):
        """Clean up resources."""
        if self.connection:
            # Close database connection
            self.connection = None

        # Remove temporary files
        for _temp_file in self.temp_files:
            # os.remove(temp_file)
            pass
        self.temp_files.clear()


async def mount_database_tool(coordinator: Any, config: dict[str, Any] | None = None) -> Callable:
    """Mount database tool with cleanup."""
    tool = DatabaseTool(config or {})
    await tool.connect()

    # Mount tool
    # await coordinator.mount("tools", tool, name="database")

    # Return cleanup function
    def cleanup():
        tool.cleanup()
        logger.info("DatabaseTool cleaned up")

    return cleanup


print("Best Practice #3: Resource Cleanup")
print("\nResources that need cleanup:")
print("  - Database connections")
print("  - File handles")
print("  - Network sockets")
print("  - Temporary files")
print("  - Background threads")
print("  - Cached data")
print("\nThe cleanup function is called:")
print("  - When the session ends")
print("  - On graceful shutdown")
print("  - Before module reload (if supported)")

## Part 9: Module Lifecycle Summary

Let's visualize the complete module lifecycle from discovery to cleanup.

In [ ]:
lifecycle = [
    (
        "1. Profile Declaration",
        [
            "Profile specifies modules with source URIs",
            "Modules declared in YAML frontmatter",
            "Each module has: module ID, source URI, config",
        ],
    ),
    (
        "2. Source Resolution (amplifierd)",
        [
            "Daemon parses profile on session creation",
            "Resolves git+ sources: clone → extract → cache",
            "Resolves file:// sources: validate path exists",
            "Resolves embedded sources: profile's modules/ dir",
        ],
    ),
    (
        "3. Mount Plan Generation",
        [
            "Create dict mapping module IDs to specs",
            "Module ID format: {profile-path}.{type}.{name}",
            "Each spec: module, source (file://), config",
            "Mount plan passed to amplifier-core",
        ],
    ),
    (
        "4. Loading (amplifier-core)",
        [
            "ModuleLoader receives mount plan",
            "Imports module from source path",
            "Retrieves mount() function",
            "Prepares coordinator and config",
        ],
    ),
    (
        "5. Mounting",
        [
            "Call mount(coordinator, config)",
            "Module creates instances",
            "Module registers with coordinator",
            "Returns cleanup function (optional)",
        ],
    ),
    (
        "6. Operation",
        [
            "Orchestrator calls module methods",
            "Tools execute actions",
            "Providers generate completions",
            "Hooks observe events",
        ],
    ),
    (
        "7. Cleanup",
        [
            "Session ending",
            "Call cleanup functions",
            "Close connections",
            "Remove temporary resources",
            "Cache remains for future sessions",
        ],
    ),
]

print("Module Lifecycle:\n" + "=" * 60)
for phase, steps in lifecycle:
    print(f"\n{phase}:")
    for step in steps:
        print(f"  → {step}")

print("\n" + "=" * 60)
print("Key Guarantees:")
print("  ✓ Failed module loading doesn't crash kernel")
print("  ✓ Modules are isolated from each other")
print("  ✓ Cleanup always runs (unless hard crash)")
print("  ✓ Protocol stability maintained")
print("  ✓ Cache persists across sessions (faster subsequent loads)")

## Part 10: Troubleshooting Common Issues

### Issue 1: Module Not Found

In [ ]:
troubleshooting_guide = {
    "Module Not Found in Profile": {
        "error": "Module 'tool-xyz' not found when creating session",
        "checks": [
            "Source URI specified in profile?",
            "Git repository accessible?",
            "Correct branch/tag specified?",
            "Module ID matches package name?",
            "Mount function exists in module?",
        ],
        "debug": [
            "# Check profile syntax",
            "cat $AMPLIFIERD_HOME/share/profiles/{collection}/{profile}/{profile}.md",
            "",
            "# Check daemon logs",
            "journalctl -u amplifierd -f",
            "",
            "# Test git clone manually",
            "git clone https://github.com/org/repo",
            "",
            "# Check cache",
            "ls -la $AMPLIFIERD_HOME/cache/git/",
        ],
    },
    "Mount Failed": {
        "error": "Failed to mount module 'tool-xyz': ...",
        "checks": [
            "mount() signature correct?",
            "No exceptions during mount?",
            "Config valid for this module?",
            "Dependencies installed in module?",
            "Source path accessible?",
        ],
        "debug": [
            "# Check mount plan (in session API response)",
            "# Look for file:// source paths",
            "",
            "# Test mount directly (development)",
            "from amplifier_core import ModuleCoordinator",
            "coordinator = ModuleCoordinator(...)",
            "await mount(coordinator, {'key': 'value'})",
            "",
            "# Check amplifier-core logs",
            "# Look for module loading errors",
        ],
    },
    "Source Resolution Failed": {
        "error": "Could not resolve source: git+https://...",
        "checks": [
            "Git URL correct and accessible?",
            "Branch/tag exists in repository?",
            "Network connectivity?",
            "Git credentials configured (if private)?",
            "Subdirectory path correct (if using #subdir)?",
        ],
        "debug": [
            "# Test git access manually",
            "git ls-remote https://github.com/org/repo",
            "",
            "# Check daemon resolution logs",
            "journalctl -u amplifierd | grep 'Resolving source'",
            "",
            "# Clear cache and retry",
            "rm -rf $AMPLIFIERD_HOME/cache/git/{commit}",
        ],
    },
    "Behavior Issues": {
        "error": "Module loaded but not working correctly",
        "checks": [
            "Protocol implementation matches exactly?",
            "Return types correct?",
            "Error handling graceful?",
            "Config passed to mount()?",
            "Logging shows what's happening?",
        ],
        "debug": [
            "# Check session events",
            "# Via API: GET /api/v1/sessions/{id}/events",
            "",
            "# Check tool execution events",
            "# Look for tool:execute events with errors",
            "",
            "# Test module in isolation",
            "# Create minimal test profile",
        ],
    },
}

print("Troubleshooting Guide:\n" + "=" * 60)
for issue, details in troubleshooting_guide.items():
    print(f"\n{issue}")
    print(f"  Error: {details['error']}")
    print("\n  Checks:")
    for check in details["checks"]:
        print(f"    □ {check}")
    print("\n  Debug steps:")
    for step in details["debug"]:
        if step:  # Skip empty lines
            print(f"    $ {step}")

## Summary

This notebook covered:

1. **amplifier-core vs amplifierd**: Core execution runtime vs REST API daemon
2. **Module Philosophy**: Linux kernel-inspired, mechanism vs policy
3. **Six Module Types**: Provider, Tool, Orchestrator, Context, Hook, Agent
4. **Module Contracts**: Protocol-based interfaces for each type
5. **Module Resolution**: Manifest-based with source URIs (git+, file://, embedded)
6. **Source Resolution**: Git cloning, caching, mount plan generation
7. **Mounting**: The mount() function and coordinator registration
8. **Creating Modules**: Step-by-step tool and provider examples
9. **Advanced Patterns**: Multiple tools per module
10. **Best Practices**: Error handling, configuration, resource cleanup
11. **Lifecycle**: From profile declaration to cleanup
12. **Troubleshooting**: Common issues and solutions

### Key Takeaways

- **Modules declared in profiles** with explicit source URIs
- **amplifierd resolves sources** (clones git repos, caches locally)
- **Mount plans use file:// paths** to cached module locations
- **amplifier-core loads modules** from mount plan specifications
- **Protocols define contracts** - implement them exactly
- **Never raise exceptions** - return errors gracefully
- **Configuration is key** - make modules flexible
- **Clean up resources** - return cleanup functions from mount()
- **One module, many tools** - group related functionality

### Module ID Format

In production: `{profile-path}.{type}.{name}`

Examples:
- `foundation/base.provider.anthropic`
- `foundation/base.tool.filesystem`
- `custom/agent.tool.database`

### Source URI Examples

```yaml
# Git repository (most common)
source: git+https://github.com/microsoft/amplifier-module-provider-anthropic@main

# With specific version
source: git+https://github.com/org/repo@v1.2.3

# With subdirectory
source: git+https://github.com/org/monorepo@main#subdir=packages/tool-xyz

# Local development
source: file:///home/user/dev/my-module

# Embedded in profile
source: embedded
```

### Directory Structure

```
$AMPLIFIERD_HOME/
├── share/
│   └── profiles/{collection}/{profile}/
│       ├── {profile}.md           # Profile with module declarations
│       ├── agents/                # Agent configs
│       ├── context/               # Context files
│       └── modules/               # Embedded modules (optional)
└── cache/
    └── git/
        ├── {commit}/              # Cached git module
        └── {commit}_{subdir}/     # Cached git subdir module
```

### Next Steps

1. Study existing modules (provider-anthropic, tool-filesystem)
2. Create your own tool module
3. Test locally with `source: file:///path/to/module`
4. Publish to GitHub
5. Use in profiles with `source: git+https://...`
6. Modules cached automatically for fast subsequent loads

### Related Documentation

- [Profiles Guide](./guides/profiles.md) - Configure modules via profiles
- [Mounts Guide](./guides/mounts.md) - Mount plan specifications
- [API Guide](./guides/api.md) - REST API for session management
- [Development Guide](./guides/development.md) - Creating custom modules